<div class="alert alert-block alert-info">
    <center><h1>ELT Project Overview: GoodReads Data</h1></center>
</div>

In [39]:
# pip install psycopg2

In [40]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

<div class="alert alert-block alert-info">
    <center><h1>CSV file imported to dataframe</h1></center>
</div>

In [41]:
data_file = "amazon.csv"

In [42]:
amazon_db1 = pd.read_csv(data_file)
amazon_db1.head()

,amazon_title,amazon_author,amazon_rating,amazon_reviews count,amazon_isbn-10,amazon_price
0,Tell Me your Dreams,by Sidney Sheldon,4.4,160.0,8172234902,209
1,The Boy in the Striped Pyjamas (Definitions),by John Boyne,4.6,134.0,1862305277,350
2,Romancing the Balance Sheet: For Anyone Who Ow...,by Anil Lamba,4.5,156.0,9350294311,477
3,Mossad,by Michael Bar-Zohar - Import,4.6,637.0,8184958455,340
4,My Story,by Kamala Das,4.5,42.0,8172238975,178


<div class="alert alert-block alert-info">
    <center><h1>Check for and dropped rows with incomplete data.</h1></center>
</div>

In [43]:
amazon_db1.count()

amazon_title            1382
amazon_author           1382
amazon_rating           1378
amazon_reviews count    1378
amazon_isbn-10          1382
amazon_price            1382
dtype: int64

In [44]:
amazon_db1 = amazon_db1.dropna(how='any')
amazon_db1.count()

amazon_title            1378
amazon_author           1378
amazon_rating           1378
amazon_reviews count    1378
amazon_isbn-10          1378
amazon_price            1378
dtype: int64

<div class="alert alert-block alert-info">
    <center><h1>Renamed column headers</h1></center>
</div>

In [45]:
renamed_df = amazon_db1.rename(columns={"amazon_title":"title", "amazon_author":"author_name", "amazon_rating":"amazon_rating", "amazon_reviews count":"amazon_ratings_count", "amazon_isbn-10":"isbn10", "amazon_price":"Price (Rs)"})
renamed_df.head()

,title,author_name,amazon_rating,amazon_ratings_count,isbn10,Price (Rs)
0,Tell Me your Dreams,by Sidney Sheldon,4.4,160.0,8172234902,209
1,The Boy in the Striped Pyjamas (Definitions),by John Boyne,4.6,134.0,1862305277,350
2,Romancing the Balance Sheet: For Anyone Who Ow...,by Anil Lamba,4.5,156.0,9350294311,477
3,Mossad,by Michael Bar-Zohar - Import,4.6,637.0,8184958455,340
4,My Story,by Kamala Das,4.5,42.0,8172238975,178


<div class="alert alert-block alert-info">
    <center><h1>Split "author name" column on "by "</h1></center>
</div>

In [46]:
renamed_df[["by", "author_name"]]= renamed_df.author_name.str.split("by ", expand = True)
# renamed_df

<div class="alert alert-block alert-info">
    <center><h1>Convertng Rupees to USD</h1></center>
</div>

In [47]:
# https://stackoverflow.com/questions/42530372/multiplying-columns-with-a-constant-value-in-pandas-type-error
renamed_df["amazon_price_usd"] = renamed_df["Price (Rs)"]*0.015
# renamed_df

<div class="alert alert-block alert-info">
    <center><h1>Split "author name"</h1></center>
</div>

In [48]:
renamed_df[["first_name", "last_name"]]= renamed_df.author_name.str.split(" ", n=1, expand = True)
renamed_df.head(3)

,title,author_name,amazon_rating,amazon_ratings_count,isbn10,Price (Rs),by,amazon_price_usd,first_name,last_name
0,Tell Me your Dreams,Sidney Sheldon,4.4,160.0,8172234902,209,,3.13,Sidney,Sheldon
1,The Boy in the Striped Pyjamas (Definitions),John Boyne,4.6,134.0,1862305277,350,,5.25,John,Boyne
2,Romancing the Balance Sheet: For Anyone Who Ow...,Anil Lamba,4.5,156.0,9350294311,477,,7.15,Anil,Lamba


<div class="alert alert-block alert-info">
    <center><h1>Removed unneeded columns</h1></center>
</div>

In [49]:
renamed2 = renamed_df.drop("by", axis=1)
renamed3 = renamed2.drop("author_name", axis=1)
renamed4 = renamed3.drop("Price (Rs)", axis=1)
renamed4.head()

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba
3,Mossad,4.6,637.0,8184958455,5.10,Michael,Bar-Zohar - Import
4,My Story,4.5,42.0,8172238975,2.67,Kamala,Das


<div class="alert alert-block alert-info">
    <center><h1>Checked for incomplete data and remove rowsl</h1></center>
</div>

In [50]:
renamed4.count()

title                   1378
amazon_rating           1378
amazon_ratings_count    1378
isbn10                  1378
amazon_price_usd        1378
first_name              1186
last_name               1139
dtype: int64

In [51]:
renamed4 = renamed4.dropna(how='any')
renamed4.count()

title                   1139
amazon_rating           1139
amazon_ratings_count    1139
isbn10                  1139
amazon_price_usd        1139
first_name              1139
last_name               1139
dtype: int64

<div class="alert alert-block alert-info">
    <center><h1>Split last name then removed labels, descriptions, and extra hyphens.</h1></center>
</div>

In [52]:
renamed5 = renamed4['last_name'].str.split(n=5, expand=True)
renamed5.columns = ['last_name{}'.format(x+1) for x in renamed5.columns]
amazon_db2 = renamed4.join(renamed5)
amazon_db2.head(11)

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name,last_name1,last_name2,last_name3,last_name4,last_name5
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon,Sheldon,None,None,None,None
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne,Boyne,None,None,None,None
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba,Lamba,None,None,None,None
3,Mossad,4.6,637.0,8184958455,5.10,Michael,Bar-Zohar - Import,Bar-Zohar,-,Import,None,None
4,My Story,4.5,42.0,8172238975,2.67,Kamala,Das,Das,None,None,None,None
5,Inspiring Thoughts (Inspiring Thoughts Quotati...,4.1,148.0,8170286840,1.80,A.P.J.,Abdul Kalam,Abdul,Kalam,None,None,None
6,12 Years a Slave,4.5,41.0,8175994479,1.48,Solomon,Northup,Northup,None,None,None,None
7,Headley And I,3.5,14.0,9350295725,3.71,S.,Hussain Zaidi,Hussain,Zaidi,None,None,None
8,Making India Awesome: New Essays and Columns,3.8,1188.0,8129137429,1.50,Chetan,Bhagat,Bhagat,None,None,None,None
9,The Dirty Dozen: Hitmen of the Mumbai Underworld,4.2,54.0,9386850133,3.31,Gabriel,Khan,Khan,None,None,None,None


In [53]:
d = {'Import': '','Unabridged':'', 'Abridged':'','Large Print':'', 'Illustrated': '', 'Publications': '', '-':''}
amazon_db2 = amazon_db2.replace(d)
amazon_db2.head(5)

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name,last_name1,last_name2,last_name3,last_name4,last_name5
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon,Sheldon,None,None,None,None
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne,Boyne,None,None,None,None
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba,Lamba,None,None,None,None
3,Mossad,4.6,637.0,8184958455,5.10,Michael,Bar-Zohar - Import,Bar-Zohar,,,None,None
4,My Story,4.5,42.0,8172238975,2.67,Kamala,Das,Das,None,None,None,None


In [54]:
amazon_db2.dtypes

title                    object
amazon_rating            object
amazon_ratings_count     object
isbn10                    int64
amazon_price_usd        float64
first_name               object
last_name                object
last_name1               object
last_name2               object
last_name3               object
last_name4               object
last_name5               object
dtype: object

<div class="alert alert-block alert-info">
    <center><h1>Removed "none" labels, remove extra columns, rejoined columns, and renamed columns.</h1></center>
</div>

In [55]:
amazon_db2.fillna(value="", inplace=True)
amazon_db2

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name,last_name1,last_name2,last_name3,last_name4,last_name5
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon,Sheldon,,,,
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne,Boyne,,,,
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba,Lamba,,,,
3,Mossad,4.6,637.0,8184958455,5.10,Michael,Bar-Zohar - Import,Bar-Zohar,,,,
4,My Story,4.5,42.0,8172238975,2.67,Kamala,Das,Das,,,,
5,Inspiring Thoughts (Inspiring Thoughts Quotati...,4.1,148.0,8170286840,1.80,A.P.J.,Abdul Kalam,Abdul,Kalam,,,
6,12 Years a Slave,4.5,41.0,8175994479,1.48,Solomon,Northup,Northup,,,,
7,Headley And I,3.5,14.0,9350295725,3.71,S.,Hussain Zaidi,Hussain,Zaidi,,,
8,Making India Awesome: New Essays and Columns,3.8,1188.0,8129137429,1.50,Chetan,Bhagat,Bhagat,,,,
9,The Dirty Dozen: Hitmen of the Mumbai Underworld,4.2,54.0,9386850133,3.31,Gabriel,Khan,Khan,,,,


In [56]:
amazon_db2 = amazon_db2.drop(columns = "last_name")
amazon_db2.head()

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name1,last_name2,last_name3,last_name4,last_name5
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon,,,,
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne,,,,
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba,,,,
3,Mossad,4.6,637.0,8184958455,5.10,Michael,Bar-Zohar,,,,
4,My Story,4.5,42.0,8172238975,2.67,Kamala,Das,,,,


In [57]:
amazon_db2['last_name'] = amazon_db2['last_name1'] + " " + amazon_db2['last_name2']+ " " + amazon_db2['last_name3']+ " " + amazon_db2['last_name4']+ " " + amazon_db2['last_name5']
amazon_db2

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name1,last_name2,last_name3,last_name4,last_name5,last_name
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon,,,,,Sheldon
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne,,,,,Boyne
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba,,,,,Lamba
3,Mossad,4.6,637.0,8184958455,5.10,Michael,Bar-Zohar,,,,,Bar-Zohar
4,My Story,4.5,42.0,8172238975,2.67,Kamala,Das,,,,,Das
5,Inspiring Thoughts (Inspiring Thoughts Quotati...,4.1,148.0,8170286840,1.80,A.P.J.,Abdul,Kalam,,,,Abdul Kalam
6,12 Years a Slave,4.5,41.0,8175994479,1.48,Solomon,Northup,,,,,Northup
7,Headley And I,3.5,14.0,9350295725,3.71,S.,Hussain,Zaidi,,,,Hussain Zaidi
8,Making India Awesome: New Essays and Columns,3.8,1188.0,8129137429,1.50,Chetan,Bhagat,,,,,Bhagat
9,The Dirty Dozen: Hitmen of the Mumbai Underworld,4.2,54.0,9386850133,3.31,Gabriel,Khan,,,,,Khan


In [58]:
amazon_db2 = amazon_db2.drop(columns = "last_name1")
amazon_db2 = amazon_db2.drop(columns = "last_name2")
amazon_db2 = amazon_db2.drop(columns = "last_name3")
amazon_db2 = amazon_db2.drop(columns = "last_name4")
amazon_db2 = amazon_db2.drop(columns = "last_name5")
amazon_db2.head(3)

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba


In [59]:
amazon_db2 = amazon_db2[amazon_db2.first_name !="Dreamland"]
amazon_db2.head()

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba
3,Mossad,4.6,637.0,8184958455,5.10,Michael,Bar-Zohar
4,My Story,4.5,42.0,8172238975,2.67,Kamala,Das


<div class="alert alert-block alert-info">
    <center><h1>Formatted the rating count.</h1></center>
</div>

In [60]:
pd.options.display.float_format = '{:,.2f}'.format
amazon_db2.head(3)

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba


In [61]:
amazon_db2["amazon_ratings_count"].astype(str).astype(float)
amazon_db2.head(3)

,title,amazon_rating,amazon_ratings_count,isbn10,amazon_price_usd,first_name,last_name
0,Tell Me your Dreams,4.4,160.0,8172234902,3.13,Sidney,Sheldon
1,The Boy in the Striped Pyjamas (Definitions),4.6,134.0,1862305277,5.25,John,Boyne
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,156.0,9350294311,7.15,Anil,Lamba


In [62]:
amazon_db2.dtypes

title                    object
amazon_rating            object
amazon_ratings_count     object
isbn10                    int64
amazon_price_usd        float64
first_name               object
last_name                object
dtype: object

In [63]:
amazon_floats = amazon_db2['amazon_ratings_count'].astype(float)
# pd.options.display.float_format = '{:,.0f}'.format
amazon_floats.head()

0   160.00
1   134.00
2   156.00
3   637.00
4    42.00
Name: amazon_ratings_count, dtype: float64

In [64]:
amazon_db2 = amazon_db2.drop(columns = "amazon_ratings_count")
amazon_db2.head(3)

,title,amazon_rating,isbn10,amazon_price_usd,first_name,last_name
0,Tell Me your Dreams,4.4,8172234902,3.13,Sidney,Sheldon
1,The Boy in the Striped Pyjamas (Definitions),4.6,1862305277,5.25,John,Boyne
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,9350294311,7.15,Anil,Lamba


In [65]:
amazon_round = amazon_floats.round(0).astype(int)

In [66]:
float_table = pd.DataFrame({"amazon_ratings_count": amazon_round})
float_table.head()

,amazon_ratings_count
0,160
1,134
2,156
3,637
4,42


<div class="alert alert-block alert-info">
    <center><h1>Rejoined formated ratings count.</h1></center>
</div>

In [67]:
amazon_db2["amazon_ratings_count"]=float_table
amazon_db2

,title,amazon_rating,isbn10,amazon_price_usd,first_name,last_name,amazon_ratings_count
0,Tell Me your Dreams,4.4,8172234902,3.13,Sidney,Sheldon,160
1,The Boy in the Striped Pyjamas (Definitions),4.6,1862305277,5.25,John,Boyne,134
2,Romancing the Balance Sheet: For Anyone Who Ow...,4.5,9350294311,7.15,Anil,Lamba,156
3,Mossad,4.6,8184958455,5.10,Michael,Bar-Zohar,637
4,My Story,4.5,8172238975,2.67,Kamala,Das,42
5,Inspiring Thoughts (Inspiring Thoughts Quotati...,4.1,8170286840,1.80,A.P.J.,Abdul Kalam,148
6,12 Years a Slave,4.5,8175994479,1.48,Solomon,Northup,41
7,Headley And I,3.5,9350295725,3.71,S.,Hussain Zaidi,14
8,Making India Awesome: New Essays and Columns,3.8,8129137429,1.50,Chetan,Bhagat,1188
9,The Dirty Dozen: Hitmen of the Mumbai Underworld,4.2,9386850133,3.31,Gabriel,Khan,54


<div class="alert alert-block alert-info">
    <center><h1>Upload to Postgres.</h1></center>
</div>

In [68]:
amazon_db2.to_csv("amazon_df2.csv", sep = ",", encoding = 'utf-8', index = False)

In [69]:
rds_connection_string = "postgres:***********@localhost:5432/Goodreads"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [70]:
engine.table_names()

['flipkart', 'goodreads']

In [71]:
amazon_db2.to_sql(name='amazon', con=engine, if_exists='append', index=True)